
# Korean MRC T5 Model



## Data Downlaod

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hyonukjo","key":"60518c9c6bb1d3cc6384176874004683"}'}

In [ ]:
!ls -1ha kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 방지
!chmod 600 ~/.kaggle/kaggle.json  

kaggle.json


In [ ]:
!kaggle competitions download -c k-digital-goorm-5-korean-mrc
!pip install transformers
!pip install sentencepiece
!pip install wandb
!gdown "14n7WhvotoTYg1yh1_hBauTjCoG_N4iC7&confirm=t" 
!mkdir model
!mkdir data

Downloading...
From: https://drive.google.com/uc?id=14n7WhvotoTYg1yh1_hBauTjCoG_N4iC7&confirm=t
To: /content/extra.zip
100% 35.2M/35.2M [00:00<00:00, 221MB/s]


In [ ]:
import os
from zipfile import ZipFile
from glob import glob 
import pandas as pd 



# 알집 경로
dir = '/content/'

# 실제로 알집이 풀어질 경로
base_dir = '/content/data/'
base_dir_1 = '/content/'


# os.chdir(base_dir)

# 알집을 풀어볼까용?
# ZipFile(dir + 'k-digital-goorm-5-korean-mrc.zip').extractall(base_dir)
ZipFile(dir + 'extra.zip').extractall(base_dir_1)



# 실제로 잘 풀렸는지 확인

os.listdir(base_dir)
os.listdir(base_dir_1)


['.config',
 'kaggle.json',
 'k-digital-goorm-5-korean-mrc.zip',
 'data',
 'extra_train5',
 'extra.zip',
 'model',
 '.ipynb_checkpoints',
 'sample_data']

## import

In [ ]:
import sentencepiece
import transformers
import numpy as np
import os
import pdb
import argparse
from dataclasses import dataclass, field
from typing import Optional
from collections import defaultdict
import nltk
import wandb
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.cuda.amp import GradScaler, autocast
from typing import List, Tuple, Dict, Any
import json
import random
import math
import re
from collections import Counter
from tqdm import tqdm, trange
import torch.nn.functional as F
from transformers import (
    T5Tokenizer, 
    T5ForConditionalGeneration,
    AutoConfig,
    AdamW
)

## 데이터셋 구성
현재 JSON 데이터를 볼 수 있는 클래스를 하나 작성하자.

In [ ]:
class Set_data:
    def __init__(self, data, indices: List[Tuple[int, int, int]]):
        self._data = data
        self._indices = indices

    # Json을 불러오는 메소드
    @classmethod
    def load(cls, file_path: str):
        with open(file_path, 'r', encoding='utf-8') as fd:
            data = json.load(fd)

        indices = []
        for d_id, document in enumerate(data['data']):
            for p_id, paragraph in enumerate(document['paragraphs']):
                for q_id, _ in enumerate(paragraph['qas']):
                    indices.append((d_id, p_id, q_id))
        
        return cls(data, indices)

    # 데이터 셋을 잘라내는 메소드
    @classmethod
    def split(cls, dataset, eval_ratio: float=.1, seed=42):
        indices = list(dataset._indices)
        random.seed(seed)
        random.shuffle(indices)
        train_indices = indices[int(len(indices) * eval_ratio):]
        eval_indices = indices[:int(len(indices) * eval_ratio)]

        return cls(dataset._data, train_indices), cls(dataset._data, eval_indices)

    def __getitem__(self, index: int) -> Dict[str, Any]:
        d_id, p_id, q_id = self._indices[index]
        paragraph = self._data['data'][d_id]['paragraphs'][p_id]

        context = paragraph['context']
        qa = paragraph['qas'][q_id]

        guid = qa['guid']
        question = qa['question']
        answers = qa['answers']

        return {
            'guid': guid,
            'context': context,
            'question': question,
            'answers': answers
        }

    def __len__(self) -> int:
        return len(self._indices)


`load` 메소드를 이용해서 Json 데이터를 불러올 수 있다.

In [ ]:
dataset = Set_data.load('./data/train.json')
print("Number of Samples:", len(dataset))
print(dataset[0])

Number of Samples: 12037
{'guid': '798db07f0b9046759deed9d4a35ce31e', 'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.', 'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?', 'answers': [{'text': '한 달가량', 'answer_start': 478}, {'text': '한 달', 'answer_start': 478}]}


`split` 메소드를 이용하면 데이터 셋을 나눌 수 있다.

In [ ]:
#데이터 속성 확인
train_dataset, dev_dataset = Set_data.split(dataset)
print("Number of Train Samples:", len(train_dataset))
print("Number of Dev Samples:", len(dev_dataset))

Number of Train Samples: 10834
Number of Dev Samples: 1203


In [ ]:
file_path3 = './extra_train5/news_train5.json'
with open(file_path3, 'r', encoding = 'utf-8-sig') as fd:
    news_train = json.load(fd)
file_path4 = './extra_train5/news_dev5.json'
with open(file_path4, 'r', encoding = 'utf-8-sig') as fd:
    news_dev = json.load(fd)
file_path5 = './extra_train5/extra_train5.json'
with open(file_path5, 'r', encoding = 'utf-8-sig') as fd:
    extra_train = json.load(fd)
file_path6 = './extra_train5/extra_dev5.json'
with open(file_path6, 'r', encoding = 'utf-8-sig') as fd:
    extra_dev = json.load(fd)

In [ ]:
print(len(news_train),len(news_dev),len(extra_train),len(extra_dev))

10000 1000 30000 3000


In [ ]:
tokenizer = T5Tokenizer.from_pretrained('KETI-AIR/ke-t5-base')
max_source_length = 1031
max_target_length = 30
padding = False

def preprocess(data):
  guid=data['guid']
  prefix='klue_mrc_gen: '
  question=data['question'].strip()
  context=data['context'].strip()
  answer=sorted(data['answers'],key=lambda x:len(x['text']))[0]['text']
  inputs=str(prefix+"klue_mrc question: "+question+" context: "+context)
  model_inputs=tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)
  labels=tokenizer(answer, max_length=max_target_length,padding=padding, truncation=True)
  model_inputs["labels"]=labels["input_ids"]
  model_inputs["example_id"]=guid
  return model_inputs
def preprocess_news(data):
  guid=data['guid']
  prefix='klue_mrc_gen: '
  question=data['question'].strip()
  context=data['context'].strip()
  answer=data['answers']['text']
  inputs=str(prefix+"klue_mrc question: "+question+" context: "+context)
  model_inputs=tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)
  labels=tokenizer(answer, max_length=max_target_length,padding=padding, truncation=True)
  model_inputs["labels"]=labels["input_ids"]
  model_inputs["example_id"]=guid
  return model_inputs

In [ ]:
preprocessed_train=[]
preprocessed_dev=[]

for i in range(len(train_dataset)):
  preprocessed_train.append(preprocess(train_dataset[i]))
for i in range(len(dev_dataset)):
  preprocessed_dev.append(preprocess(dev_dataset[i]))

In [ ]:
# for i in range(len(news_train)):
#   preprocessed_train.append(preprocess_news(news_train[i][0]))
# for i in range(len(news_dev)):
#   preprocessed_dev.append(preprocess_news(news_dev[i][0]))
for i in range(len(extra_train)):
  preprocessed_train.append(preprocess(extra_train[i][0]))
for i in range(len(extra_dev)):
  preprocessed_dev.append(preprocess(extra_dev[i][0]))

In [ ]:
print(len(preprocessed_train), len(preprocessed_dev))

40834 4203


In [ ]:
def collate_fn_style(samples):
    input_ids, labels, attention_mask=[],[],[]
    for i in range(len(samples)):
      input_ids.append(torch.tensor(samples[i]['input_ids']))
      labels.append(torch.tensor(samples[i]['labels']))
      attention_mask.append(torch.tensor(samples[i]['attention_mask']))
      
    input_ids=pad_sequence(input_ids, batch_first=True)
    labels = pad_sequence(labels, batch_first=True)
    attention_mask=pad_sequence(attention_mask, batch_first=True)

    return input_ids, labels, attention_mask

In [ ]:
accumulation=16
train_batch_size=32
eval_batch_size=16

train_loader = torch.utils.data.DataLoader(preprocessed_train,
                                           batch_size=train_batch_size//accumulation,
                                           shuffle=True, collate_fn=collate_fn_style,
                                           pin_memory=True, num_workers=2)
dev_loader = torch.utils.data.DataLoader(preprocessed_dev, batch_size=eval_batch_size//accumulation,
                                         shuffle=False, collate_fn=collate_fn_style,
                                         num_workers=2)

In [ ]:
random_seed=42
np.random.seed(random_seed)
torch.manual_seed(random_seed)
model_name = 'KETI-AIR/ke-t5-base'
model=T5ForConditionalGeneration.from_pretrained(model_name)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(64128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(64128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [ ]:
model.train()
learning_rate = 1e-4
optimizer = AdamW(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
#평가 metric들
from scipy import spatial
import wandb
def compute_edit_acc(predictions, target_labels):
    return nltk.edit_distance(predictions, target_labels)
def compute_cosine_acc(predictions, target_labels):
    predictions_list=list(predictions)
    target_labels_list=list(target_labels)
    distance_pred=[]
    distance_tar=[]
    if len(predictions)>=len(target_labels):
        len_dif=len(predictions)-len(target_labels)
        for i in range(len(target_labels_list)):
            if target_labels_list[i]==predictions_list[i]:
                distance_pred.append(1)
                distance_tar.append(1)
            else:
                distance_pred.append(0)
                distance_tar.append(1)
        distance_tar.extend([1]*len_dif)
        distance_pred.extend([0]*len_dif)
              
    else:
        len_dif=len(target_labels)-len(predictions)
        for i in range(len(predictions_list)):
            if target_labels_list[i]==predictions_list[i]:
                distance_pred.append(1)
                distance_tar.append(1)
            else:
                distance_pred.append(0)
                distance_tar.append(1)
        distance_tar.extend([1]*len_dif)
        distance_pred.extend([0]*len_dif)

    return 1 - spatial.distance.cosine(distance_pred, distance_tar)

In [ ]:
# wandb 이용법
'''
1. wandAPI keys 복사해서 로그인
2. wandb.config에 자신의 훈련 환경 기록
3. 그대로 훈련 돌리기. 추후 project 이름은 변경 가능.
4. run name 변경해서, 중복 방지하기
4. iteration은 기록의 일관성을 위해 수정하지 않아야 함.
'''
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
wandb.init(project="project name", entity="wxxk2")
wandb.run.name = "korean_mrc_T5_20230110_bs-128_epoch-5_lr-3e-4_Mickey_savemodel"

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: wxxk2. Use `wandb login --relogin` to force relogin


In [ ]:
wandb.config = {
  "learning_rate": 1e-4,
  "epochs": 5,
  "batch_size": 16
}

In [ ]:
# cuda 지우기
import torch
torch.cuda.empty_cache()

In [ ]:
# cuda 지우기
import gc
gc.collect()

0

In [ ]:
train_epoch = 5
lowest_valid_loss = 9999.
accuracy=[]
validation_loss=[]

for epoch in range(train_epoch):
    with tqdm(train_loader, unit="batch") as tepoch:
        optimizer.zero_grad()
        for iteration, (input_ids, labels, attention_mask) in enumerate(tepoch):
            tepoch.set_description(f"Epoch {epoch}")
            input_ids = input_ids.to(device)
            labels = labels.to(device)
            attention_mask=attention_mask.to(device)


            output = model(input_ids=input_ids,
                           labels=labels)

            loss = output.loss
            loss=loss/accumulation
            loss.backward()
            if ((iteration + 1) % accumulation == 0) or (iteration + 1 == len(tepoch)):
                optimizer.step()
                optimizer.zero_grad()

                tepoch.set_postfix(loss=loss.item())

            if iteration != 0 and iteration % int(len(train_loader) / 2) == 0:
                edit_acc=0
                cosine_acc=0
                cnt=0
                with torch.no_grad():
                    model.eval()
                    valid_losses = []
                    predictions = []
                    target_labels = []
                    for input_ids, labels, attention_mask in tqdm(dev_loader,desc='Eval', position=1, leave=None):
                        input_ids = input_ids.to(device)
                        labels = labels.to(device)
                        attention_mask=attention_mask.to(device)

                        output = model(input_ids=input_ids,
                                       labels=labels)
                        
                        logits = output.logits
                        loss = output.loss
                        valid_losses.append(loss.item())
                        
                        batch_predictions = []
                        for i in range(len(logits)):
                          cur_batch=[]
                          for j in range(len(logits[i])):
                            cur_batch.append(logits[i][j].argmax(dim=-1))
                          batch_predictions.append(torch.tensor(cur_batch))

                        batch_labels = [example for example in labels]
                        decoded_preds=tokenizer.batch_decode(batch_predictions, skip_special_tokens=True)
                        decoded_labels=tokenizer.batch_decode(batch_labels, skip_special_tokens=True)
                        for i in range(len(decoded_labels)):
                            edit_acc+=compute_edit_acc(decoded_preds[i],decoded_labels[i])
                            cosine_acc+=compute_cosine_acc(decoded_preds[i],decoded_labels[i])
                            cnt+=1

                # edit_acc = edit_acc/cnt (조호준,카이스트,자연어처리,캠프,피드백 받기전)           
                edit_acc += edit_acc/cnt
                cosine_acc=cosine_acc/cnt
                valid_loss = sum(valid_losses) / len(valid_losses)
                wandb.log({"validation loss": valid_loss, "edit_acc": edit_acc, "cosine_acc":cosine_acc})
                wandb.watch(model)
                if lowest_valid_loss > valid_loss:
                    lowest_valid_loss=valid_loss
                    print('Edit Acc for model which have lower valid loss: ', edit_acc)
                    print('Cosine Acc for model which have lower valid loss: ', cosine_acc)

                    torch.save(model.state_dict(), "./model/best_model_20221018.bin")
                    torch.save(model.state_dict(), "./model/pytorch_model_20221018.bin")
                else:
                    print("Valid loss increased.")
                    print('Edit Acc for model which have lower valid loss: ', edit_acc)
                    print('Cosine Acc for model which have lower valid loss: ', cosine_acc)

                    torch.save(model.state_dict(), "./model/pytorch_model_20221018.bin")

Eval:   0%|          | 0/4203 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/scipy/spatial/distance.py:699: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)

Eval: 100%|█████████▉| 4201/4203 [03:39<00:00, 20.39it/s]
                                                         

Edit Acc for model which have lower valid loss:  27665.58077563645
Cosine Acc for model which have lower valid loss:  0.9994153078017457


Eval: 100%|█████████▉| 4201/4203 [03:38<00:00, 21.32it/s]
                                                         

Edit Acc for model which have lower valid loss:  27543.55174875089
Cosine Acc for model which have lower valid loss:  0.9291942371535581


Eval: 100%|█████████▉| 4201/4203 [03:39<00:00, 20.62it/s]
                                                         

...

Eval: 100%|█████████▉| 4201/4203 [03:46<00:00, 20.82it/s]
                                                         

Edit Acc for model which have lower valid loss:  12338.935046395432
Cosine Acc for model which have lower valid loss:  0.9183723666217493


Eval: 100%|██████████| 4203/4203 [03:44<00:00, 19.91it/s]
                                                         

Edit Acc for model which have lower valid loss:  12249.913871044491
Cosine Acc for model which have lower valid loss:  0.9152203418420668


Epoch 4: 100%|██████████| 20417/20417 [1:43:49<00:00,  3.28batch/s, loss=0.168]


## Test 출력 파일 작성

In [ ]:
test_dataset = Set_data.load('./data/test.json')
preprocessed_test=[]

In [ ]:
max_source_length = 1031
max_target_length = 30
padding = False

def preprocess_test(data):
  guid=data['guid']
  prefix='klue_mrc_gen: '
  question=data['question'].strip()
  context=data['context'].strip()
  inputs=str(prefix+"klue_mrc question: "+question+" context: "+context)
  model_inputs=tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)
  model_inputs["guid"]=guid
  return model_inputs
preprocessed_test=[]
for i in range(len(test_dataset)):
  preprocessed_test.append(preprocess_test(test_dataset[i]))

In [ ]:
def collate_fn_style_test(samples):
    input_ids=[]
    attention_mask=[]
    for i in range(len(samples)):
      input_ids.append(torch.tensor(samples[i]['input_ids']))
      attention_mask.append(torch.tensor(samples[i]['attention_mask']))
    input_ids=pad_sequence(input_ids, batch_first=True)
    attention_mask=pad_sequence(attention_mask, batch_first=True)
    
    return input_ids, attention_mask

In [ ]:
test_batch_size = 1
test_loader = torch.utils.data.DataLoader(preprocessed_test, batch_size=test_batch_size,
                                          shuffle=False, collate_fn=collate_fn_style_test,
                                          num_workers=2)

In [ ]:
with torch.no_grad():
    model.eval()
    predictions = []
    answers=[]
    cnt=0
    for input_ids, attention_mask in tqdm(test_loader, desc='Test', position=1, leave=None):

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        output = model.generate(input_ids=input_ids,do_sample=True, max_length=30, top_k=50, top_p=0.75, early_stopping=True, no_repeat_ngram_size=2)
        
        # output = model(input_ids=input_ids, decoder_input_ids=input_ids)

        # logits = output.logits

        # batch_predictions = []
        # for i in range(len(logits)):
        #   cur_batch=[]
        #   for j in range(len(logits[i])):
        #     cur_batch.append(logits[i][j].argmax(dim=-1))
        #   batch_predictions.append(torch.tensor(cur_batch))

        decoded_preds=tokenizer.batch_decode(output, skip_special_tokens=True)
        for i in range(len(decoded_preds)):
            answers.append([decoded_preds[i]])

In [ ]:
Id=[]
Predicted=[]
for i in range(len(test_dataset)):
    answer_dict={}
    Id.append(test_dataset[i]['guid'])
    Predicted.append(answers[i][0])

In [ ]:
for i in range(500,550):
  print(test_dataset[i]['question'])
  print(Predicted[i])

In [ ]:
import pandas as pd
df = pd.DataFrame({'Id': Id,'Predicted':Predicted})
df.to_csv('submit.csv', index=False,encoding="utf-8-sig")

In [ ]:
while True:pass